In [3]:
import pandas as pd
import numpy as np
import os
import re

In [4]:
date = "20210709_204655"
directory = f'_output/hedges/pnl/{date}'

In [5]:
PNL = "PNL__KDE__DeltaGammaHedge__BULLISH__3000__30.csv"

In [6]:
prices = pd.read_csv("_output/hedges/pnl/20210709_204655/__prices.csv")
prices = dict(prices[["label", "price"]].to_numpy())

In [7]:
rows = []

# stats = ['mean', 'std', 'skew', 'kurt', '1%', '25%', '50%', '75%', '99%']
stats = ['mean', 'std', 'skew', 'kurt', '1%', '99%']

for fname in [fname for fname in os.listdir(directory) if fname.endswith('.csv') and fname.startswith('PNL')]:
    paths, model, hedge, period, strike, maturity = re.match("PNL__(\w+)__(\w+)__(\w+)__(\w+)__(\w+)__(\w+)__100000.csv", fname).groups()
    df = pd.read_csv(f"{directory}/{fname}", header=None)
    label = re.match(".*_(\w+__\d+__\d+)__\d+.csv$", fname).groups()[0]
    price = prices[label]
    quantile_low = df.quantile(0.01)
    quantile = df.quantile(.99)
    skew = df.skew()
    kurt = df.kurtosis()
    df = df.describe()
    df = df.T
    df['skew'] = skew
    df['kurt'] = kurt
    df['1%'] = quantile_low
    df['99%'] = quantile
    df = np.round(df/price, 3)
    del df['count']
    del df['min']
    del df['max']
    df = df[stats]
    rows += [[paths, model, hedge, period, strike, maturity, *df.to_numpy()[0]]]
    
columns = ['paths', 'model', 'hedge', 'period', 'strike', 'maturity', *stats]
df = pd.DataFrame(rows, columns=columns)

# df = df[df['maturity'] == '90']
df = df[df['strike'].isin(['8367', '9804', '4088'])]
df

,paths,model,hedge,period,strike,maturity,mean,std,skew,kurt,1%,99%
5,SVCJ,HESTON,DeltaGammaHedge,COVID,9804,30,-0.118,0.587,-0.006,0.104,-2.048,1.242
6,KDE,SVJ,DeltaHedge,BULLISH,4088,30,-0.301,0.632,0.001,0.009,-1.960,1.476
7,SVCJ,MERTON,MinimumVarianceHedge,COVID,9804,30,-0.075,1.329,-0.004,0.010,-5.127,1.733
8,KDE,BLACK_SCHOLES,DeltaHedge,CALM,8367,90,0.466,0.136,0.000,0.000,0.131,0.796
10,SVCJ,SVJ,DeltaGammaHedge,COVID,9804,30,-0.135,0.605,-0.004,0.079,-2.161,1.282
...,...,...,...,...,...,...,...,...,...,...,...,...
809,SVCJ,SVCJ,DeltaVegaHedge,COVID,9804,30,-0.127,0.639,-0.002,0.085,-2.370,1.280
810,KDE,VARIANCE_GAMMA,DeltaHedge,BULLISH,4088,30,-0.302,0.634,0.001,0.009,-1.973,1.484
814,SVCJ,BLACK_SCHOLES,DeltaGammaHedge,CALM,8367,30,0.014,0.438,0.003,0.043,-1.216,1.250
820,KDE,MERTON,DeltaHedge,CALM,8367,30,0.432,0.260,-0.000,0.003,-0.296,1.120


In [12]:
hs = {
    'DeltaHedge': 0,
    'DeltaGammaHedge': 1,
    'DeltaVegaHedge': 2,
    'MinimumVarianceHedge': 3,
}

ms = {
    'BLACK_SCHOLES': 0,
    'HESTON': 1,
    'VARIANCE_GAMMA': 2,
    'SVJ': 3,
    'SVCJ': 4,
    'CGMY': 5,
    'MERTON': 6,
}

hn = {
    'DeltaHedge': "\Delta ",
    'DeltaGammaHedge': "\Delta-\Gamma ",
    'DeltaVegaHedge': "\Delta-\mathcal V ",
    'MinimumVarianceHedge': "\textbf{MV} ",
}

mn = {
    'BLACK_SCHOLES': "BS",
    'HESTON': "SV",
    'VARIANCE_GAMMA': "VG",
    'SVJ': 'SVJ',
    'SVCJ': 'SVCJ',
    'CGMY': 'CGMY',
    'MERTON': 'Merton',
}

for (paths, period), odf in df.groupby(['maturity', 'period']):
    print("# ", paths, period, "\n\n")
    odf1 = odf[odf['paths'] == 'SVCJ']
    odf2 = odf[odf['paths'] == 'KDE']
    odf = pd.merge(odf1, odf2, on=['maturity', 'model', 'hedge', 'period', 'strike'], suffixes=('_SVCJ', '_KDE'))
    odf['hs'] = odf.apply(lambda x: hs[x['hedge']], axis=1)
    odf['ms'] = odf.apply(lambda x: ms[x['model']], axis=1)
    odf['name'] = odf.apply(lambda x: f"${hn[x['hedge']]}_{{{mn[x['model']]}}}$", axis=1)
    odf = np.round(odf, 3)
    odf = odf.sort_values(['hs', 'ms'])[['name', *[f'{x}_SVCJ' for x in stats], *[f'{x}_KDE' for x in stats]]]
#     odf.columns = ['name', 'mean', 'std', 'skew', 'kurt', '1\%', '25\%', '50\%', '75\%', '99\%']
    caption = f"Summary statistics of {period} {paths} hedges"
    label = f"tab:{paths}_{period}"
    print(f"\\begin{{table}}\n\centering\n{odf.to_latex(index=False, escape=False)}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}")
    print('\n==================================================================\n\n')

#  30 BULLISH 




KeyError: "['mean_90', 'kurt_90', 'skew_30', '99%_30', 'std_90', 'skew_90', 'std_30', '99%_90', 'kurt_30', '1%_90', 'mean_30', '1%_30'] not in index"